### 필요한 모듈 불러오기

In [ ]:
import os
import pandas as pd

from assistant import ChatAssistant
from utils import create_pdf

### 인물 및 참조 파일(PDF, TXT) 지정

PDF: 해당 인물에 대한 나무위키 문서 \
TXT: 해당 인물의 말투 학습을 위한 대본

코드 및 파일 일관성을 위해 pdf 파일 이름은 `"인물이름.pdf"`,\
대본 파일 이름은 `"인물이름_script.txt"`로 통일해주세요!

In [ ]:
datapath = "./data"
name = "천지훈"

if f"{name}.pdf" not in os.listdir(datapath):
    create_pdf(name)  # 동명이인이 존재할경우 name, full_url 모두 입력

pdf_file_name = os.path.join(datapath, f"{name}.pdf")
txt_file_name = os.path.join(datapath, f"{name}_script.txt")

### PDFAssistant 정의

OpenAI API의 어시스턴트 기능을 활용하여 PDF를 바탕으로 대화하는 기능 구현 

`instruction`: 인물 정보를 기반으로 언어 모델이 어떤 역할을 수행해야 하는지에 대한 지시사항

아래 코드를 참고하여 각자의 방법으로 커스터마이징해서 사용

In [ ]:
instructions = f"""
제공된 정보를 이용하여 {name}의 특징과 말투를 모방한 챗봇을 제작해야 합니다. 
출력은 해당 인물의 확립된 성격 및 말투와 일치해야 합니다.
해당 인물의 본질을 유지하면서 일관성 있고 상황에 맞는 응답을 보장해야 합니다. 
해당 인물의 스타일에 일관되게 어울리는 페르소나를 사용하세요. 
해당 인물의 사고 과정에 따라 응답을 구성하기 위해 연쇄 사고(CoT)를 통합합니다.
스크립트의 예시와 함께 멀티샷 프롬프트를 적용하여 GPT 모델을 안내합니다.
해당 인물 내 반응을 장려하는 긍정적인 안내를 포함하세요.
"""

In [ ]:
client = ChatAssistant(name="천지훈")
# 스크립트 없이 나무위키 PDF 만으로 만들고 싶은 경우
# client.set_assistant(instructions=instructions, pdf_file=pdf_file_name) 
client.set_assistant(instructions=instructions, pdf_file=pdf_file_name, txt_file=txt_file_name) 

### 스크립트 파일 수정 예시

같은 방법으로 PDF 파일도 변경하거나, 둘 다 변경할 수 있음\
`new_pdf_file_name`을 새로 선언하고, `change_file(pdf_file=new_pdf_file_name)`과 같이 pdf 파일 이름을 넘겨주면 됩니다.

In [ ]:
# new_txt_file_name = os.path.join(datapath, f"{name}_new_script.txt")

# client.change_file(txt_file=new_txt_file_name)

In [ ]:
print(client.get_instructions())

# 💡 Instruction Tuning
> 프롬프트 엔지니어링을 위해서는 아래의 프롬프트를 자유롭게 수정하면서 테스트하면 됩니다~

`revise_instructions` 함수 실행 후에는 반드시! `get_instructions`를 통해 수정이 되었는지 확인해주세요.

In [ ]:
new_instruction=f"""
제공된 PDF 파일과 스크립트를 바탕으로 가상의 인물을 생성하세요. 문서에서 발견된 텍스트, 스타일, 어조 및 특정 언어적 특성을 분석하여 개인의 말하기, 생각하기, 표현 방식을 이해합니다. 이 인물은 문서에서 나타난 개인의 언어 패턴, 어휘력, 그리고 특이점을 정확하게 반영해야 합니다. 스크립트가 제공되면, 인물은 문서에서 알려진 커뮤니케이션 방식과 일관되게 스타일리시하고 어조가 있게 텍스트를 해석하고 전달해야 합니다. 인물은 스크립트의 내용과 맥락에 적응하면서, 개인의 독특한 목소리의 진정성과 본질을 유지하며 상호 작용하는 동안에도 일관되게 반응하도록 해야 합니다.

앞으로는 주어진 인물인 {name}의 입장에서 응답하세요.
"""
client.revise_instructions(new_instruction)

In [ ]:
print(client.get_instructions())

아래 셀을 실행하면 무한 루프가 실행되는데, 대화를 그만하려면 `exit`나 `quit`를 입력해서 종료!

In [ ]:
q_list = []
a_list = []

while True:
        question = input("Enter your question (or type 'exit' to quit): ")
        if question.lower() in ['exit', 'quit']:
            break

        answers = client.get_answers(question)
        for answer in answers:
            print(answer) 
        q_list.append(question)
        a_list.append(answers)

### 💡 반드시 아래 셀을 실행해서 실험 내용들을 저장하고 관리해주세요!

In [ ]:
import time
timestamp = str(time.time()).split()[0]

generated_df = pd.DataFrame(
    {
        "질문": q_list,
        "답변": a_list
    }
)

pd.set_option('display.max_colwidth', None)
generated_df.to_csv(f"{name}_test_{timestamp}", index=False)

generated_df